# Using Null

- teacher

id	| dept	| name	| phone	| mobile
----|-------|-------|-------|-----
101	| 1 | Shrivell	| 2753 | 07986 555 1234
102	| 1	| Throd	    | 2754 | 07122 555 1920
103	| 1	| Splint	| 2293	|
104 |	| Spiregrain | 3287	|
105 | 2	| Cutflower	 | 3212 | 07996 555 6574
106 |	| Deadyawn | 3345 |	
... |      |        |        |

- dept

id	| name
----|----
1	| Computing
2	| Design
3	| Engineering
... |

### Teachers and Departments
The school includes many departments. Most teachers work exclusively for a single department. Some teachers have no department.

[Selecting NULL values](https://sqlzoo.net/wiki/Selecting_NULL_values).

In [1]:
library(tidyverse)
library(DBI)
library(getPass)
drv <- switch(Sys.info()['sysname'],
             Windows="PostgreSQL Unicode(x64)",
             Darwin="/usr/local/lib/psqlodbcw.so",
             Linux="PostgreSQL")
con <- dbConnect(
  odbc::odbc(),
  driver = drv,
  Server = "localhost",
  Database = "sqlzoo",
  UID = "postgres",
  PWD = getPass("Password?"),
  Port = 5432
)
options(repr.matrix.max.rows=20)

─ Attaching packages ──────────────────── tidyverse 1.3.0 ─

✔ ggplot2 3.3.0     ✔ purrr   0.3.4
✔ tibble  3.0.1     ✔ dplyr   0.8.5
✔ tidyr   1.0.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

─ Conflicts ───────────────────── tidyverse_conflicts() ─
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



Password? ····


## 1. NULL, INNER JOIN, LEFT JOIN, RIGHT JOIN

List the teachers who have NULL for their department.

> _Why we cannot use =_   
> You might think that the phrase dept=NULL would work here but it doesn't - you can use the phrase dept IS NULL
> 
> _That's not a proper explanation._  
> No it's not, but you can read a better explanation at Wikipedia:NULL.

In [2]:
teacher <- dbReadTable(con, 'teacher')
dept <- dbReadTable(con, 'dept')

In [3]:
teacher %>% 
    filter(is.na(dept)) %>%
    select(name)

name
<chr>
Spiregrain
Deadyawn


## 2.
Note the INNER JOIN misses the teachers with no department and the departments with no teacher.

In [4]:
teacher %>% 
    inner_join(dept, by=c(dept="id")) %>%
    select(name.x, name.y) %>%
    rename(teacher=name.x, dept=name.y)

teacher,dept
<chr>,<chr>
Shrivell,Computing
Throd,Computing
Splint,Computing
Cutflower,Design


## 3.
Use a different JOIN so that all teachers are listed.

In [5]:
teacher %>%
    left_join(dept, by=c(dept="id")) %>%
    select(name.x, name.y) %>% 
    rename(teacher=name.x, dept=name.y)

teacher,dept
<chr>,<chr>
Shrivell,Computing
Throd,Computing
Splint,Computing
Spiregrain,NA
Cutflower,Design
Deadyawn,NA


## 4.
Use a different JOIN so that all departments are listed.

In [6]:
teacher %>% 
    right_join(dept, by=c(dept="id")) %>%
    select(name.x, name.y) %>%
    rename(teacher=name.x, dept=name.y)

teacher,dept
<chr>,<chr>
Shrivell,Computing
Throd,Computing
Splint,Computing
Cutflower,Design
NA,Engineering


## 5. Using the [COALESCE](https://sqlzoo.net/wiki/COALESCE) function


Use COALESCE to print the mobile number. Use the number '07986 444 2266' if there is no number given. **Show teacher name and mobile number or '07986 444 2266'**

In [7]:
teacher %>% mutate(
    mobile=replace_na(mobile, '07986 444 2266')) %>%
    select(name, mobile)

name,mobile
<chr>,<chr>
Shrivell,07986 555 1234
Throd,07122 555 1920
Splint,07986 444 2266
Spiregrain,07986 444 2266
Cutflower,07996 555 6574
Deadyawn,07986 444 2266


## 6.
Use the COALESCE function and a LEFT JOIN to print the teacher name and department name. Use the string 'None' where there is no department.

In [8]:
teacher %>%
    left_join(dept, by=c(dept="id")) %>%
    select(name.x, name.y) %>%
    rename(teacher=name.x, dept=name.y) %>%
    mutate(dept=replace_na(dept, 'None'))

teacher,dept
<chr>,<chr>
Shrivell,Computing
Throd,Computing
Splint,Computing
Spiregrain,None
Cutflower,Design
Deadyawn,None


## 7.
Use COUNT to show the number of teachers and the number of mobile phones.

In [9]:
as_tibble(c(
    teacher %>% 
    tally(!is.na(.$name), name='n_name'), 
    teacher %>% 
    tally(!is.na(.$mobile), name='n_mobile'))
)

n_name,n_mobile
<int>,<int>
6,3


## 8.
Use COUNT and GROUP BY **dept.name** to show each department and the number of staff. Use a RIGHT JOIN to ensure that the Engineering department is listed.

In [10]:
teacher %>% 
    right_join(dept, by=c(dept="id")) %>%
     count(name.y, wt=!is.na(name.x))

name.y,n
<chr>,<int>
Computing,3
Design,1
Engineering,0


## 9. Using [CASE](https://sqlzoo.net/wiki/CASE)


Use CASE to show the **name** of each teacher followed by 'Sci' if the teacher is in **dept** 1 or 2 and 'Art' otherwise.

In [11]:
teacher %>% 
    left_join(dept, by=c(dept="id")) %>%
    mutate(flag=case_when(
        dept %in% 1:2 ~ 'Sci', 
        TRUE          ~ 'Art')) %>%
    select(name.x, flag)

name.x,flag
<chr>,<chr>
Shrivell,Sci
Throd,Sci
Splint,Sci
Spiregrain,Art
Cutflower,Sci
Deadyawn,Art


## 10.
Use CASE to show the name of each teacher followed by 'Sci' if the teacher is in dept 1 or 2, show 'Art' if the teacher's dept is 3 and 'None' otherwise.

In [12]:
teacher %>% 
    left_join(dept, by=c(dept="id")) %>%
    mutate(flag=case_when(
        dept %in% 1:2 ~ 'Sci',
        dept == 3     ~ 'Art',
        TRUE          ~ 'None')) %>%
    select(name.x, flag)

name.x,flag
<chr>,<chr>
Shrivell,Sci
Throd,Sci
Splint,Sci
Spiregrain,None
Cutflower,Sci
Deadyawn,None


In [13]:
dbDisconnect(con)